## Introduction
About 100.000 users that watched in total 17.770 movies;
- Each user watched between 300 and 3000 movies
- The file contains about 65.000.000 records (720 MB) of the form:
`<user_id, movie_id> : “user_id watched movie_id”`
- Similarity between users: Jaccard similarity of sets of movies they watched: 
``` 
jsim(S1, S2) = #intersect(S1, S2)/#union(S1, S2)
``` 
- Task: find (with help of LSH) pairs of users whose jsim > 0.5

Process:
1. Tune it (signature length, number of bands, number of rows per band)
2. Randomize, optimize, benchmark, polish the code, ...
3. Dump results to a text file ans.txt (just a csv list of records: user1, user2) 

## Data preparation

In [25]:
# import packages
import numpy as np
import pandas as pd
import time
import csv
from collections import defaultdict
from scipy.sparse import csc_matrix

t0 = time.time()
np.random.seed(seed=100)

In [26]:
#  load the data
FILE = '../data/user_movie.npy'
df = pd.DataFrame(np.load(FILE), columns = ['user','movie'])

m_by_u = df.groupby('user')['movie'].apply(list)

## MinHash

The dataset is so small that random permutations is used instead of hash functions. 
In this way we can avoid time consuming loops.

In [27]:
# set user as the column, movie as the row in the sparse matrix
mat = csc_matrix(([1]*df.shape[0], (df.iloc[:,1], df.iloc[:,0])))

In [28]:
n_movie = mat.shape[0]
n_user = mat.shape[1]

In [29]:
# Relatively short signatures (50-150) should result in good results (and take less time to compute).
n_sig = 150

In [30]:
def minhashing(mat, n_sig = n_sig):
    # create the sig matrix using minhashing
    sig_matrix = np.zeros([n_sig, n_user])
    for i in range(n_sig):
        perm = mat[np.random.permutation(n_movie)]
        for j in range(n_user):
            sig_matrix[i,j] = int(perm.indices[perm.indptr[j]:perm.indptr[j+1]].min())
        if i%10==9: # check the progress
            print(n_sig - i - 1,' signatures remaining to be created')
    return sig_matrix
sig_matrix = minhashing(mat,n_sig)

140  signatures remaining to be created
130  signatures remaining to be created
120  signatures remaining to be created
110  signatures remaining to be created
100  signatures remaining to be created
90  signatures remaining to be created
80  signatures remaining to be created
70  signatures remaining to be created
60  signatures remaining to be created
50  signatures remaining to be created
40  signatures remaining to be created
30  signatures remaining to be created
20  signatures remaining to be created
10  signatures remaining to be created
0  signatures remaining to be created


In [31]:
cal_time = round(time.time()-t0)
print("From data importing to minhashing, it takes {0:.2f} minutes".format(cal_time/60))

From data importing to minhashing, it takes 6.28 minutes


In [32]:
# check the dimension of the sig matrix
sig_matrix.shape

(150, 103703)

In [33]:
#np.savetxt('sig.txt', sig_matrix, fmt='%d')
#sig_matrix = np.loadtxt('sig.txt', dtype=int)

## Implement the LSH algorithm

When there are many users that fall into the same bucket (i.e., there are many candidates for being similar to each other) then checking if all the potential pairs are really similar might be very expensive: you have to check k(k-1)/2 pairs, when the bucket has k elements. Postpone evaluation of such a bucket till the very end (or just ignore it – they are really expensive). Or better: consider increasing the number of rows per band – that will reduce the chance of encountering big buckets.

Note that b*r doesn’t have to be exactly the length of the signature. For example, when you work with signature of length n=100, you may consider e.g., b=3, r=33, b=6, r=15, etc. – any combination of b
 
To make sure that your program will not exceed the 30 minutes runtime you are advised to close the result.txt file after any new pair is appended to it (and open it again, when you want to append a new one).

In [34]:
def check_estimate_pair(pair,n_sig):
    # check if the pair from the bucket is similar based on sig matrix
    sigi = sig_matrix[:,pair[0]]#.tolist()#.ravel().tolist()[0]
    sigj = sig_matrix[:,pair[1]]#.tolist()#.ravel().tolist()[0]
    # estimate_jsim
    intersect = np.count_nonzero(sigi == sigj)
    jsim = intersect/n_sig
    if jsim > 0.40:
        return True

def check_true_pair(pair):
    # check if the similar pair based on sig matrix is a true pair
    a = m_by_u[pair[0]]
    b = m_by_u[pair[1]]
    intersect = len(set(a) & set(b))
    union = len(a) + len(b) - intersect
    jsim = intersect/union
    if jsim > 0.5:
        return True

Too many slices would give a lot of false positives while too few slices would only be able to identify the highest degrees of similarity.

In [35]:
bands = 25
rows = 6
max_bucket_item = 300

### note
bands = 30
rows = 5
max_bucket_item = 300
Unique pairs:  21482770
Similar pairs based on sig matrix:  878065
True similar pairs:  639



## Check, no write out

In [36]:
def LSH_bucket(sig_matrix, max_bucket_item = 25):
    unique_pairs = set() 
    for b in range(bands):
        bucket_list = defaultdict(set) # remove the duplicate items
        s = np.sum(sig_matrix[b*rows:(b+1)*rows,:], axis =0)
        for index, x in np.ndenumerate(s):
            bucket_list[x].add(index[0])
        for key, value in bucket_list.items():
            if len(value) > 1 and len(value) < max_bucket_item: 
                for i in value:
                    for j in value:
                        if i < j and ((i,j) not in unique_pairs): # remove the duplicate pair
                            unique_pairs.add((i,j))
    return unique_pairs

In [37]:
def LSH_bucket_2(sig_matrix, max_bucket_item = 25):
    unique_pairs = set() 
    for b in range(bands):
        bucket_list = defaultdict(set) # remove the duplicate items
        df = pd.DataFrame(sig_matrix[b*rows:(b+1)*rows,:])
        h = df.apply(lambda x: hash(tuple(x)), axis = 0)
        for index, x in np.ndenumerate(h):
            bucket_list[x].add(index[0])
        for key, value in bucket_list.items():
            if len(value) > 1 and len(value) < max_bucket_item: 
                for i in value:
                    for j in value:
                        if i < j and ((i,j) not in unique_pairs): # remove the duplicate pair
                            unique_pairs.add((i,j))
    return unique_pairs

In [38]:
def print_not_write():
    unique_pairs = LSH_bucket_2(sig_matrix,max_bucket_item)
    print('Unique pairs: ',len(unique_pairs))

    sim_pairs = []
    for pair in unique_pairs:
        if check_estimate_pair(pair,n_sig):
            sim_pairs.append(pair)
    print('Similar pairs based on sig matrix: ',len(sim_pairs))

    true_pairs = []
    for pair in sim_pairs:
        if check_true_pair(pair):
            true_pairs.append(pair) 
    print('True similar pairs: ', len(true_pairs))
#print_not_write()

## Write to text file, for final submital

In [39]:
def LSH_bucket_true_paris(sig_matrix, max_bucket_item = 25): 
    unique_pairs = set() 
    for b in range(bands):
        bucket_list = defaultdict(set) # remove the duplicate items
        df = pd.DataFrame(sig_matrix[b*rows:(b+1)*rows,:])
        h = df.apply(lambda x: hash(tuple(x)), axis = 0)
        for index, x in np.ndenumerate(h):
            bucket_list[x].add(index[0])
        for key, value in bucket_list.items():
            if len(value) > 1 and len(value) < max_bucket_item: 
                for i in value:
                    for j in value:
                        if i < j and ((i,j) not in unique_pairs): # remove the duplicate pair
                            unique_pairs.add((i,j))
                            if check_estimate_pair((i,j),n_sig):
                                if check_true_pair((i,j)):
                                    with open('ans.txt','a') as file:
                                        writer = csv.DictWriter(file,fieldnames=['user1', 'user2'])
                                        writer.writerow({'user1':i,'user2':j})
LSH_bucket_true_paris(sig_matrix,max_bucket_item)

## Total Time

In [40]:
cal_time = round(time.time()-t0)
print("Total running time is {0:.2f} minutes".format(cal_time/60))

Total running time is 13.45 minutes


Reference:
- [hash tuple](https://stackoverflow.com/questions/25757042/create-hash-value-for-each-row-of-data-with-selected-columns-in-dataframe-in-pyt/25757564)
- [hash string](https://www.pythoncentral.io/hashing-strings-with-python/)